In [ ]:
import torch
from torch import nn, optim
import torch.nn as nn
import sys
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
from fastapi import FastAPI
from fastapi import File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from io import BytesIO
sys.path.append('/content/drive/MyDrive/Skripsi/Script/Bullying10K/')
from readdata import *
import timeit

In [ ]:
class C3D(nn.Module):
    """
    The C3D network.
    """

    def __init__(self, num_classes, pretrained=True, inchannel=3,step=16):
        super(C3D, self).__init__()

        self.conv1 = nn.Conv3d(inchannel, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))
        if step==16:
            self.fc6 = nn.Linear(8192, 4096)
            self.fc7 = nn.Linear(4096, 4096)
            self.fc8 = nn.Linear(4096, num_classes)
        elif step==10:
            self.fc6 = nn.Linear(8192, 4096)
            self.fc7 = nn.Linear(4096, 4096)
            self.fc8 = nn.Linear(4096, num_classes)
        elif step==4:
            self.fc6 = nn.Linear(8192, 4096)
            self.fc7 = nn.Linear(4096, 4096)
            self.fc8 = nn.Linear(4096, num_classes)


        self.dropout = nn.Dropout(p=0.5)

        self.relu = nn.ReLU()

        self.__init_weight()

        if pretrained:
            self.__load_pretrained_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        x = self.relu(self.conv2(x))
        x = self.pool2(x)

        x = self.relu(self.conv3a(x))
        x = self.relu(self.conv3b(x))
        x = self.pool3(x)

        x = self.relu(self.conv4a(x))
        x = self.relu(self.conv4b(x))
        x = self.pool4(x)

        x = self.relu(self.conv5a(x))
        x = self.relu(self.conv5b(x))
        x = self.pool5(x)

        x = x.view(-1, 8192)
        x = self.relu(self.fc6(x))
        x = self.dropout(x)
        x = self.relu(self.fc7(x))
        x = self.dropout(x)

        logits = self.fc8(x)

        return logits

    def __load_pretrained_weights(self):
        """Initialiaze network."""
        corresp_name = {
                        # Conv1
                        "features.0.weight": "conv1.weight",
                        "features.0.bias": "conv1.bias",
                        # Conv2
                        "features.3.weight": "conv2.weight",
                        "features.3.bias": "conv2.bias",
                        # Conv3a
                        "features.6.weight": "conv3a.weight",
                        "features.6.bias": "conv3a.bias",
                        # Conv3b
                        "features.8.weight": "conv3b.weight",
                        "features.8.bias": "conv3b.bias",
                        # Conv4a
                        "features.11.weight": "conv4a.weight",
                        "features.11.bias": "conv4a.bias",
                        # Conv4b
                        "features.13.weight": "conv4b.weight",
                        "features.13.bias": "conv4b.bias",
                        # Conv5a
                        "features.16.weight": "conv5a.weight",
                        "features.16.bias": "conv5a.bias",
                         # Conv5b
                        "features.18.weight": "conv5b.weight",
                        "features.18.bias": "conv5b.bias",
                        # fc6
                        "classifier.0.weight": "fc6.weight",
                        "classifier.0.bias": "fc6.bias",
                        # fc7
                        "classifier.3.weight": "fc7.weight",
                        "classifier.3.bias": "fc7.bias",
                        }

        # p_dict = torch.load("C:\\Users\\isalo\\Documents\\Skripsi\\Program\\Backend\\c3d-pretrained.pth")
        s_dict = self.state_dict()
        for name in p_dict:
            if name not in corresp_name:
                continue
            s_dict[corresp_name[name]] = p_dict[name]
        self.load_state_dict(s_dict)

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

def get_1x_lr_params(model):
    """
    This generator returns all the parameters for conv and two fc layers of the net.
    """
    b = [model.conv1, model.conv2, model.conv3a, model.conv3b, model.conv4a, model.conv4b,
         model.conv5a, model.conv5b, model.fc6, model.fc7]
    for i in range(len(b)):
        for k in b[i].parameters():
            if k.requires_grad:
                yield k

def get_10x_lr_params(model):
    """
    This generator returns all the parameters for the last fc layer of the net.
    """
    b = [model.fc8]
    for j in range(len(b)):
        for k in b[j].parameters():
            if k.requires_grad:
                yield k


def model_predict(file):
    torch.manual_seed(42)
    np.random.seed(42)
    data = readnplong(file) # return npy array loileoks like pickle load

    data=[i for i  in data]
    loc=np.random.choice(len(data)-30*4+1)
    data=data[loc:loc+30*4 :4]
    frame=np.zeros((len(data),2,260,346),dtype=float)

    for i in range(len(data)):
        frame=event2frame(data[i],i,frame)

    data=frame
    data = data.reshape(2, 30, 260, 346)
    frame=train_transform(data)

    outputs = c3d(frame.unsqueeze(0))
    probs = nn.Softmax(dim=1)(outputs)
    # preds_tensor = torch.max(probs, 1)[1]
    confidence, prediction_tensor = torch.max(probs, dim=1)
    prediction = ""
    if confidence.item() > 0.9:
      if prediction_tensor.item() == 0:
        prediction = "slapping"
      elif prediction_tensor.item() == 1:
        prediction = "kicking"
      elif prediction_tensor.item() == 2:
        prediction = "punching"
      elif prediction_tensor.item() == 3:
        prediction = "pushing"
      elif prediction_tensor.item() == 4:
        prediction = "strangling"
      elif prediction_tensor.item() == 5:
        prediction = "hairgrabs"

      return prediction
    else:
      return prediction

In [ ]:
def resize_tensor(x, size):
    return F.interpolate(x, size=[size, size], mode='bilinear', align_corners=True)

def preprocess_input(x):
    return torch.tensor(x, dtype=torch.float)

def apply_resize(x, size):
    return resize_tensor(x, size)

def resize_and_preprocess(x, size):
    x = preprocess_input(x)
    x = apply_resize(x, size)
    # print(x.shape)
    return x

def transform(x):
    return resize_and_preprocess(x, 112)

train_transform = transforms.Compose([
    transform,
])

test_transform = transforms.Compose([
    transform,
])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
c3d = C3D(num_classes=6, pretrained=False,inchannel=2)
train_params = [{'params': get_1x_lr_params(c3d), 'lr': 1e-5},
                {'params': get_10x_lr_params(c3d), 'lr': 1e-5 * 10}]
optimizer = optim.Adam(train_params,lr=1e-5, weight_decay=1e-4)
c3d.to(device)

checkpoint = torch.load('/content/drive/MyDrive/Skripsi/c3d.pth', weights_only=True, map_location=torch.device('cpu'))
c3d.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [ ]:
# %%writefile main.py

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/upload")
async def upload(file: UploadFile):
    file_content = await file.read()
    file_stream = BytesIO(file_content)
    prediction = model_predict(file_stream)

    return JSONResponse(content={"filename": file.filename, "preds":str(prediction)})


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2objlYMWyeFYSA1SAbZqcLtc890_3Ab5n3uUwH5oqAZKKb1V8"

# Set the authtoken
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [245]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://1830-34-19-14-171.ngrok-free.app
INFO:     103.80.236.168:0 - "GET / HTTP/1.1" 200 OK
INFO:     149.154.161.200:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     103.80.236.168:0 - "GET /upload HTTP/1.1" 405 Method Not Allowed
INFO:     103.47.133.65:0 - "POST /upload HTTP/1.1" 200 OK